In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
import os
from xarray.coders import CFDatetimeCoder

In [2]:
coder = CFDatetimeCoder(use_cftime=True)

In [10]:
# data = Path('/Volumes/LaCie/CMIP6/CMIP/NCAR/CESM2-WACCM')
# dir = 'historical/r1i1p1f1/Omon/thetao/gn/v20190808'

data = Path('~/')
dir = 'Downloads'
filename = 'so_Omon_CESM2-WACCM_ssp126_r1i1p1f1_gn_210101-215012.nc'
file = xr.open_dataset(data / dir / filename)
file

/Users/jonniebarnsley/opt/anaconda3/envs/phd/lib/python3.11/site-packages/xarray/conventions.py:204: SerializationWarning: variable 'so' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


<xarray.Dataset> Size: 18GB
Dimensions:    (time: 600, lev: 60, nlat: 384, nlon: 320, d2: 2, vertices: 4)
Coordinates:
    lat        (nlat, nlon) float64 983kB ...
  * lev        (lev) float64 480B 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    lon        (nlat, nlon) float64 983kB ...
  * nlat       (nlat) int32 2kB 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
  * nlon       (nlon) int32 1kB 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
  * time       (time) object 5kB 2101-01-15 13:00:00.000008 ... 2150-12-15 12...
Dimensions without coordinates: d2, vertices
Data variables:
    so         (time, lev, nlat, nlon) float32 18GB ...
    time_bnds  (time, d2) object 10kB ...
    lat_bnds   (nlat, nlon, vertices) float32 2MB ...
    lon_bnds   (nlat, nlon, vertices) float32 2MB ...
    lev_bnds   (lev, d2) float32 480B ...
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   735110.0
    branch_time_in_parent:  735110.0
    case_id:                1678
    ...                     ...
    sub_experiment_id:      none
    table_id:               Omon
    tracking_id:            hdl:21.14100/85dd2a78-d8bf-44cd-99c8-b914c77289db
    variable_id:            so
    variant_info:           CMIP6 CESM2 future scenario SSP1-2.6, years 2100-...
    variant_label:          r1i1p1f1

In [11]:
def convert_lev(ds_in):
    """Convert pressure levels from hPa to m."""
    ds = xr.Dataset()
    # copy temperature and/or salinity
    for var in ['so', 'thetao']:
        if var in ds_in:
            ds[var] = ds_in[var]
    
    # convert lev coordinates
    ds.coords['lev'] = 0.01 * ds_in.coords['lev']
    ds.lev.attrs['units'] = 'm'
    ds.lev.attrs['bounds'] = 'lev_bnds'

    # convert lev_bnds
    lev_bnds = ds_in.lev_bnds.values
    ds.coords['lev_bnds'] = (('lev', 'bnds'), lev_bnds)
    ds.lev_bnds.attrs['units'] = 'm'

    return ds

In [12]:
minLat = file.lat.min(dim='nlon')
mask = minLat <= -48.
yIndices = np.nonzero(mask.values)[0]
ds = file.isel(nlat=yIndices)
ds = ds.rename({'d2': 'bnds'})
ds = convert_lev(ds)
ds

<xarray.Dataset> Size: 3GB
Dimensions:   (nlat: 59, nlon: 320, time: 600, lev: 60, bnds: 2)
Coordinates:
    lat       (nlat, nlon) float64 151kB -79.22 -79.22 -79.22 ... -48.24 -48.24
    lon       (nlat, nlon) float64 151kB ...
  * nlat      (nlat) int32 236B 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59
  * nlon      (nlon) int32 1kB 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
  * time      (time) object 5kB 2101-01-15 13:00:00.000008 ... 2150-12-15 12:...
  * lev       (lev) float64 480B 5.0 15.0 25.0 ... 4.875e+03 5.125e+03 5.375e+03
    lev_bnds  (lev, bnds) float32 480B 0.0 10.0 10.0 ... 5.25e+03 5.5e+03
Dimensions without coordinates: bnds
Data variables:
    so        (time, lev, nlat, nlon) float32 3GB ...

In [13]:
annual = ds.groupby('time.year').mean('time', keep_attrs=True)
annual

<xarray.Dataset> Size: 227MB
Dimensions:   (year: 50, lev: 60, nlat: 59, nlon: 320, bnds: 2)
Coordinates:
    lat       (nlat, nlon) float64 151kB -79.22 -79.22 -79.22 ... -48.24 -48.24
    lon       (nlat, nlon) float64 151kB 320.6 321.7 322.8 ... 317.2 318.3 319.4
  * nlat      (nlat) int32 236B 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59
  * nlon      (nlon) int32 1kB 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
  * lev       (lev) float64 480B 5.0 15.0 25.0 ... 4.875e+03 5.125e+03 5.375e+03
    lev_bnds  (lev, bnds) float32 480B 0.0 10.0 10.0 ... 5.25e+03 5.5e+03
  * year      (year) int64 400B 2101 2102 2103 2104 2105 ... 2147 2148 2149 2150
Dimensions without coordinates: bnds
Data variables:
    so        (year, lev, nlat, nlon) float32 227MB nan nan nan ... 34.82 34.82

In [22]:
renamed = annual.rename({'year': 'time'})
renamed['time'] = annual.year.values*365.
renamed

<xarray.Dataset> Size: 227MB
Dimensions:   (time: 50, lev: 60, nlat: 59, nlon: 320, bnds: 2)
Coordinates:
    lat       (nlat, nlon) float64 151kB -79.22 -79.22 -79.22 ... -48.24 -48.24
    lon       (nlat, nlon) float64 151kB 320.6 321.7 322.8 ... 317.2 318.3 319.4
  * nlat      (nlat) int32 236B 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59
  * nlon      (nlon) int32 1kB 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
  * lev       (lev) float64 480B 5.0 15.0 25.0 ... 4.875e+03 5.125e+03 5.375e+03
    lev_bnds  (lev, bnds) float32 480B 0.0 10.0 10.0 ... 5.25e+03 5.5e+03
  * time      (time) float64 400B 7.669e+05 7.672e+05 ... 7.844e+05 7.848e+05
Dimensions without coordinates: bnds
Data variables:
    so        (time, lev, nlat, nlon) float32 227MB nan nan nan ... 34.82 34.82

In [46]:
ds = ds.groupby('time.year').mean('time', keep_attrs=True)

In [47]:
ds = ds.rename({'year': 'time'})
ds['time'] = 365.0*ds.time

# set attributes
cftime_attrs = {
    "units": "days since 0000-01-01 00:00:00",
    "calendar": "noleap",
}

ds.time.attrs.update(cftime_attrs)

In [48]:
ds = ds.rename({'d2': 'bnds'})
timeBounds = np.zeros((ds.sizes['time'], ds.sizes['bnds']))
timeBounds[:, 0] = ds.time.values
timeBounds[:, 1] = ds.time.values + 365
ds['time_bnds'] = (('time', 'bnds'), timeBounds)
ds['time_bnds'].attrs.update(cftime_attrs)

In [49]:
from xarray import Dataset
def decode_vars(ds: Dataset, *varnames: str):
    """Decodes time and bnds from int to datetime using CFDatetimeCoder"""
    def _decode(varname):
        var = ds.variables[varname]
        return coder.decode(var, var.attrs)

    coder = CFDatetimeCoder(use_cftime=True)
    return [_decode(varname) for varname in varnames]

In [50]:
time, bnds = decode_vars(ds, 'time', 'time_bnds')

In [18]:
time_var = ds.variables['time']
bnds_var = ds.variables['time_bnds']
decoded_time = coder.decode(time_var, time_var.attrs)
decoded_bnds = coder.decode(bnds_var, bnds_var.attrs)

In [51]:
ds = ds.assign_coords(time=time, time_bnds=bnds)

In [52]:
ds.time_bnds.values

array([[cftime.DatetimeNoLeap(1850, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.DatetimeNoLeap(1851, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
       [cftime.DatetimeNoLeap(1851, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.DatetimeNoLeap(1852, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
       [cftime.DatetimeNoLeap(1852, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.DatetimeNoLeap(1853, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
       [cftime.DatetimeNoLeap(1853, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.DatetimeNoLeap(1854, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
       [cftime.DatetimeNoLeap(1854, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.DatetimeNoLeap(1855, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
       [cftime.DatetimeNoLeap(1855, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.DatetimeNoLeap(1856, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
       [cftime.DatetimeNoLeap(1856, 1, 1, 0, 0, 0, 0, has_year_zero=True),
        cftime.Date

In [56]:
ds.lev_bnds.values

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan,

In [5]:
file = xr.open_dataset('/Users/jonniebarnsley/Downloads/so_annual_CESM2-WACCM_ssp126_r1i1p1f1_210101-215012.nc')
file

<xarray.Dataset> Size: 227MB
Dimensions:    (time: 50, lev: 60, nlat: 59, nlon: 320, bnds: 2, year: 50)
Coordinates:
    lat        (nlat, nlon) float64 151kB ...
    lon        (nlat, nlon) float64 151kB ...
  * nlat       (nlat) int32 236B 1 2 3 4 5 6 7 8 9 ... 52 53 54 55 56 57 58 59
  * nlon       (nlon) int32 1kB 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
  * lev        (lev) float64 480B 5.0 15.0 25.0 ... 5.125e+03 5.375e+03
    lev_bnds   (lev, bnds) float32 480B ...
    time       (year) object 400B ...
  * year       (year) int64 400B 2101 2102 2103 2104 ... 2147 2148 2149 2150
    time_bnds  (time, bnds) object 800B ...
Dimensions without coordinates: bnds
Data variables:
    so         (time, lev, nlat, nlon) float32 227MB ...

In [7]:
file.time.values

array([cftime.DatetimeNoLeap(2101, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2102, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2103, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2104, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2105, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2106, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2107, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2108, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2109, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2110, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2111, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2112, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2113, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2114, 1, 